In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import tqdm

# 1. [준비] 데이터 딱 하나(One Batch)만 가져오기
# train_loader_global에서 첫 번째 배치만 꺼내서 고정합니다.
sample_batch = next(iter(train_loader_global))
fixed_x, fixed_y = sample_batch
fixed_x = fixed_x.to(device).float()
fixed_y = fixed_y.to(device).float()

# === 여기 추가 ===
fixed_x = fixed_x[0:1]   # (1, 120, C)
fixed_y = fixed_y[0:1]   # (1, 10,  C)
print(f"fixed_x shape: {fixed_x.shape}, fixed_y shape: {fixed_y.shape}")

print(f"🧪 Sanity Check Start!")
print(f"   > Input Shape: {fixed_x.shape}")
print(f"   > Target Shape: {fixed_y.shape}")

# 2. 모델 & 설정 초기화 (회귀 모드)
# (Monster 또는 Ultimate Configs 사용)
model = TimeLLM(configs) 
model.to(device).float()
model.train()

# 학습률을 좀 높게 잡아서 빨리 외우게 시킵니다.
optimizer = optim.AdamW(model.parameters(), lr=0.001) 
criterion = nn.MSELoss()

# 3. [반복 학습] 똑같은 문제만 100번 풀기
epochs = 500
losses = []

print("🔥 Overfitting on one batch...")
for epoch in range(epochs):
    optimizer.zero_grad()
    
    # 똑같은 fixed_x만 계속 넣음
    B, Seq, C = fixed_x.shape
    Pred = fixed_y.shape[1]
    
    dummy_mark_enc = torch.zeros(B, Seq, 4).to(device).float()
    dummy_mark_dec = torch.zeros(B, Pred, 4).to(device).float()
    dummy_dec_in = torch.zeros(B, Pred, C).to(device).float()
    
    outputs = model(fixed_x, dummy_mark_enc, dummy_dec_in, dummy_mark_dec)
    if isinstance(outputs, tuple): outputs = outputs[0]
    
    # 0번 채널(종가)만 비교
    preds = outputs[:, -configs.pred_len:, 0]
    true = fixed_y[:, :, 0]
    
    loss = criterion(preds, true)
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if (epoch+1) % 20 == 0:
        print(f"Epoch {epoch+1}/{epochs} - Loss: {loss.item():.8f}")

# 4. [눈으로 확인] 그래프 그리기
print("\n👀 Visualizing Results...")
model.eval()
with torch.no_grad():
    outputs = model(fixed_x, dummy_mark_enc, dummy_dec_in, dummy_mark_dec)
    if isinstance(outputs, tuple): outputs = outputs[0]

# 배치 내의 첫 번째 샘플만 그려봅니다.
idx = 0
pred_seq = outputs[idx, -configs.pred_len:, 0].cpu().numpy()
true_seq = fixed_y[idx, :, 0].cpu().numpy()
input_seq = fixed_x[idx, -30:, 0].cpu().numpy() # 과거 30일

plt.figure(figsize=(12, 5))

# Loss 감소 추이
plt.subplot(1, 2, 1)
plt.plot(losses)
plt.title("Training Loss (Should go to 0)")
plt.xlabel("Epochs")
plt.ylabel("MSE Loss")
plt.grid(True)

# 예측 결과 비교
plt.subplot(1, 2, 2)
plt.plot(range(30), input_seq, label='History', color='gray')
plt.plot(range(30, 40), true_seq, label='Target', marker='o', color='black')
plt.plot(range(30, 40), pred_seq, label='Prediction', marker='x', linestyle='--', color='red')
plt.title("Prediction Visualization")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import torch
import torch.nn as nn

class DirectionalMSELoss(nn.Module):
    def __init__(self, direction_weight=5.0):
        super().__init__()
        self.mse = nn.MSELoss()
        self.weight = direction_weight

    def forward(self, pred, true, input_last_val):
        """
        pred: (B,)  예측된 내일 close_log
        true: (B,)  실제 내일 close_log
        input_last_val: (B,) 오늘 close_log (기준점)
        """
        # 1) 기본 MSE
        loss_mse = self.mse(pred, true)

        # 2) 방향 페널티
        diff_pred = pred - input_last_val   # 모델이 본 방향
        diff_true = true - input_last_val   # 실제 방향

        direction_match = diff_pred * diff_true   # 같은 방향이면 >0, 반대면 <0
        # 반대 방향(음수)일 때만 페널티
        loss_direction = torch.mean(torch.relu(-1.0 * direction_match))

        return loss_mse + self.weight * loss_direction
